In [218]:
import pandas as pd
import numpy as np
import fastparquet

In [219]:
df_raw = pd.read_csv("../../data/philadelphia_2016.csv")

In [220]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655058 entries, 0 to 655057
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   start_time          655058 non-null  object 
 1   end_time            655058 non-null  object 
 2   start_station_id    655048 non-null  float64
 3   end_station_id      655058 non-null  int64  
 4   bike_id             655058 non-null  int64  
 5   user_type           655058 non-null  object 
 6   end_station_name    655058 non-null  object 
 7   start_station_name  655048 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 40.0+ MB


You can see from the df_raw.info(), that for start and end station id and name there are 10 missing rows, otherwise there aren't any, in addition the that start_station_id is a float when in reality it should be int, due to having no decimal place.
Also the times are not formatted in the proper datetime format.

In [221]:
#Only take the rows that are not na in name or id
df = df_raw[df_raw['start_station_id'].notna() & df_raw['start_station_name'].notna()].copy()
df['start_station_id'] = df['start_station_id'].astype('int64')

#Format to Datetime
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])

In [222]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 655048 entries, 0 to 655057
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   start_time          655048 non-null  datetime64[ns]
 1   end_time            655048 non-null  datetime64[ns]
 2   start_station_id    655048 non-null  int64         
 3   end_station_id      655048 non-null  int64         
 4   bike_id             655048 non-null  int64         
 5   user_type           655048 non-null  object        
 6   end_station_name    655048 non-null  object        
 7   start_station_name  655048 non-null  object        
dtypes: datetime64[ns](2), int64(3), object(3)
memory usage: 45.0+ MB


You can see now, that there are no null values and everything is formatted correctly

In [223]:
df['duration'] = (df['end_time'] - df['start_time'])

print(f"Shortest trip: {df['duration'].min()}")
print(f"Longest trip: {df['duration'].max()}")

Shortest trip: -1 days +23:04:00
Longest trip: 19 days 00:59:00


In [224]:
#Remove all the invalid times
df = df[(df['duration'] <= pd.Timedelta("1d")) & (df['duration'] >= pd.Timedelta("1m")) | ((df['duration'] <= pd.Timedelta("5m")) & (df['start_station_id'] == df['end_station_id']))]
#Also Remove Station 3000 Called "Virtual Station" supposedly for Test Trips
df = df[(df['start_station_id'] != 3000) & (df['end_station_id'] != 3000)]

In [225]:
#Look at different Types
df['user_type'].unique()

array(['Indego30', 'Walk-up', 'IndegoFlex'], dtype=object)

In [226]:
df_stations = pd.read_parquet('../../data/stations.parquet')
del df_stations['name']

In [227]:
df = df.merge(df_stations, left_on='start_station_id', right_on='id')
del df['id']
df['start_lat'] = df['lat']
del df['lat']
df['start_lon'] = df['lon']
del df['lon']

df = df.merge(df_stations, left_on='end_station_id', right_on='id')
del df['id']
df['end_lat'] = df['lat']
del df['lat']
df['end_lon'] = df['lon']
del df['lon']


In [228]:
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [229]:
df["distance"] = haversine(
    df["start_lat"],
    df["start_lon"],
    df["end_lat"],
    df["end_lon"],
)

print(f"Smallest distance: {df['distance'].min()} km")
print(f"Greatest distance: {df['distance'].max()} km")

Smallest distance: 0.0 km
Greatest distance: 15.114165029636501 km


Calculate Speed to filter out unrealistic values. The maximum allowed speed of an ebike in the US is 20 mph, assuming the person has done no stops and as we are calculating the airline, these trips are almost certain to be faulty

In [230]:
df["speed"] = df["distance"] / df["duration"].apply(
    lambda duration: duration.total_seconds() / (60 * 60)
)

max_allowed_kmh = 20 * 1.60934 # 20mp/h in km/h

df = df[df['speed'] < max_allowed_kmh]

In [231]:
df['average_time'] = df['end_time'] - (df['duration'] / 2)

In [232]:
#Because we have weather datapoints missing, we are merging to the nearest existing weather datapoint
df_weather = pd.read_parquet("../../data/weather_hourly_philadelphia_cleaned.parquet")




In [233]:
df_weather= df_weather.reset_index()



In [234]:
df_weather.head()

,date_time,max_temp,min_temp,precip
0,2016-01-01 00:00:00,6.7,6.7,0.0
1,2016-01-01 01:00:00,7.2,7.2,0.0
2,2016-01-01 02:00:00,6.7,6.7,0.0
3,2016-01-01 03:00:00,6.7,6.7,0.0
4,2016-01-01 04:00:00,6.1,6.1,0.0


In [235]:
# df_weather datetime column was already transformed in the weather notebook, trying to do it again causes error
# df_weather['date_time'] = pd.to_datetime(df_weather['date_time'])

# df_weather = df_weather.sort_values('date_time')


In [236]:
df = df.sort_values('average_time')

df_weather_and_trips = pd.merge_asof(df, df_weather, left_on='average_time', right_on='date_time', direction='nearest')

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,end_station_name,start_station_name,duration,start_lat,start_lon,end_lat,end_lon,distance,speed,average_time,date_time,max_temp,min_temp,precip
0,2016-01-01 00:04:00,2016-01-01 00:14:00,3046,3041,3564,Indego30,"Girard Station, MFL",2nd & Market,0 days 00:10:00,39.949895,-75.143749,39.969241,-75.136180,1.005648,6.033888,2016-01-01 00:09:00,2016-01-01 00:00:00,6.7,6.7,0.0
1,2016-01-01 00:07:00,2016-01-01 00:29:00,3028,3019,2629,Indego30,6th & Race,4th & Bainbridge,0 days 00:22:00,39.940735,-75.149370,39.954694,-75.149496,0.397798,1.084905,2016-01-01 00:18:00,2016-01-01 00:00:00,6.7,6.7,0.0
2,2016-01-01 00:14:00,2016-01-01 00:27:00,3007,3066,3430,Indego30,19th & Lombard,"11th & Pine, Kahn Park",0 days 00:13:00,39.945123,-75.159952,39.945664,-75.173272,1.480278,6.832050,2016-01-01 00:20:30,2016-01-01 00:00:00,6.7,6.7,0.0
3,2016-01-01 00:20:00,2016-01-01 00:30:00,3045,3028,3715,Indego30,4th & Bainbridge,13th & Locust,0 days 00:10:00,39.947947,-75.162361,39.940735,-75.149370,1.458133,8.748797,2016-01-01 00:25:00,2016-01-01 00:00:00,6.7,6.7,0.0
4,2016-01-01 00:20:00,2016-01-01 00:42:00,3005,3064,3351,Walk-up,"18th & Washington, Chew Playground","Welcome Park, NPS",0 days 00:22:00,39.947383,-75.144145,39.938280,-75.173873,3.313616,9.037136,2016-01-01 00:31:00,2016-01-01 01:00:00,7.2,7.2,0.0
5,2016-01-01 00:21:00,2016-01-01 00:42:00,3005,3064,3355,Walk-up,"18th & Washington, Chew Playground","Welcome Park, NPS",0 days 00:21:00,39.947383,-75.144145,39.938280,-75.173873,3.313616,9.467475,2016-01-01 00:31:30,2016-01-01 01:00:00,7.2,7.2,0.0
6,2016-01-01 00:26:00,2016-01-01 00:40:00,3005,3013,2636,Walk-up,6th & Fairmount,"Welcome Park, NPS",0 days 00:14:00,39.947383,-75.144145,39.963535,-75.147818,0.615070,2.636013,2016-01-01 00:33:00,2016-01-01 01:00:00,7.2,7.2,0.0
7,2016-01-01 00:27:00,2016-01-01 00:49:00,3046,3061,3344,Walk-up,23rd & Market,2nd & Market,0 days 00:22:00,39.949895,-75.143749,39.954151,-75.177827,3.788847,10.333219,2016-01-01 00:38:00,2016-01-01 01:00:00,7.2,7.2,0.0
8,2016-01-01 00:26:00,2016-01-01 00:51:00,3046,3061,3720,Walk-up,23rd & Market,2nd & Market,0 days 00:25:00,39.949895,-75.143749,39.954151,-75.177827,3.788847,9.093232,2016-01-01 00:38:30,2016-01-01 01:00:00,7.2,7.2,0.0
9,2016-01-01 00:28:00,2016-01-01 00:49:00,3046,3061,2497,Walk-up,23rd & Market,2nd & Market,0 days 00:21:00,39.949895,-75.143749,39.954151,-75.177827,3.788847,10.825277,2016-01-01 00:38:30,2016-01-01 01:00:00,7.2,7.2,0.0


In [238]:
del df['average_time']

In [239]:
df_weather_and_trips.to_parquet('../../data/bike_trips_cleaned.parquet')